# POC: Integration of Proce Mining aand LLMs

This notebook connects to Google BigQuery to extract clinical event data, applies PM4PY to discover the Directly-Follows Graph (DFG), visualizes it, and uses LangChain with GPT to analyze and summarize the process.

## 1. Setup and Imports

This script uses libraries for cloud data access (google.cloud.bigquery), process mining and visualization (pm4py), environment management (dotenv), data handling (pandas), and language model orchestration (langchain and langchain_openai).

In [71]:
# Import libraries for cloud data access
from google.cloud import bigquery
from google_auth_oauthlib.flow import InstalledAppFlow

In [72]:
# Import libraries for process mining and visualization
from pm4py.objects.log.util import dataframe_utils
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from pm4py.visualization.dfg import visualizer as dfg_vis

In [73]:
# Import libraries for environment management
import os
from dotenv import load_dotenv

In [74]:
# Import libraries for data handling
import pandas as pd

In [75]:
# Import libraries for large language model orchestration
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

## 2. Authentication Setup

Before accessing Google Cloud services, a service account must be created in the Google Cloud Console, with the necessary IAM roles (e.g., BigQuery Admin) assigned to it. The service account’s JSON key file is securely stored locally, and its path is set using the GOOGLE_APPLICATION_CREDENTIALS environment variable to enable programmatic authentication.

In [ ]:
# Define BigQuery access scope
SCOPES = ["https://www.googleapis.com/auth/cloud-platform"]

# Run OAuth flow in the browser
flow = InstalledAppFlow.from_client_secrets_file(
    '/Users/alejandromateocobo/Documents/PythonProjects/Integration_Of_LLMs_And_Process_Mining/keys/client_secret_316641064865-57id3o26obibotvs226jeevisjdujha5.apps.googleusercontent.com.json',  # Path to the OAuth client JSON you downloaded
    scopes=SCOPES
)

# Launch browser-based login
credentials = flow.run_local_server(port=0)

## 3. Query BigQuery

Google Cloud Platform (GCP) is a suite of cloud computing services that enables scalable storage, processing, and data analysis using Google’s infrastructure. To use GCP for analyzing clinical datasets like MIMIC-III, users must create a Google account, set up a GCP project with billing, enable the necessary APIs, and configure OAuth client authentication to securely access cloud resources. The MIMIC-III database, which contains detailed health records from over 40,000 critical care patients, can be accessed through Google BigQuery for efficient cloud-based analysis, which is the recommended method by the MIT Lab for Computational Physiology.

In [ ]:
# Create BigQuery client with OAuth credentials
client = bigquery.Client(credentials=credentials, project="integration-of-pm-and-llms")

# Query MIMIC-III Clinical Dataset
query = """
    SELECT subject_id, hadm_id, admittime, dischtime, admission_type
    FROM `physionet-data.mimiciii_clinical.admissions`
    LIMIT 10
"""

df = client.query(query).to_dataframe()

df.head()

## 4. Event Log Creation

For the purpose of this Minimum Viable Product (MVP), the MIMIC-III clinical demo dataset was downloaded in CSV format, and the event log was created using Power Query in Excel by combining multiple event tables.

<details>
<summary>Power Query Script</summary>

```powerquery
let
    // Load all tables
    chartevents = Excel.CurrentWorkbook(){[Name="chartevents"]}[Content],
    datetimeevents = Excel.CurrentWorkbook(){[Name="datetimeevents"]}[Content],
    inputevents_mv = Excel.CurrentWorkbook(){[Name="inputevents_mv"]}[Content],
    outputevents = Excel.CurrentWorkbook(){[Name="outputevents"]}[Content],
    procedureevents_mv = Excel.CurrentWorkbook(){[Name="procedureevents_mv"]}[Content],
    microbiologyevents = Excel.CurrentWorkbook(){[Name="microbiologyevents"]}[Content],
    d_items = Excel.CurrentWorkbook(){[Name="d_items"]}[Content],

    // ---- CHARTEVENTS ----
    chartevents_renamed = Table.RenameColumns(chartevents, {{"charttime", "event_timestamp"}}),
    chartevents_merged = Table.NestedJoin(chartevents_renamed, {"itemid"}, d_items, {"itemid"}, "d_items", JoinKind.LeftOuter),
    chartevents_expanded = Table.ExpandTableColumn(chartevents_merged, "d_items", {"label", "linksto"}, {"label", "linksto"}),
    chartevents_selected = Table.SelectColumns(chartevents_expanded, {"itemid", "subject_id", "hadm_id", "icustay_id", "event_timestamp", "label", "linksto"}),

    // ---- DATETIMEEVENTS ----
    datetimeevents_renamed = Table.RenameColumns(datetimeevents, {{"value", "event_timestamp"}}),
    datetimeevents_merged = Table.NestedJoin(datetimeevents_renamed, {"itemid"}, d_items, {"itemid"}, "d_items", JoinKind.LeftOuter),
    datetimeevents_expanded = Table.ExpandTableColumn(datetimeevents_merged, "d_items", {"label", "linksto"}, {"label", "linksto"}),
    datetimeevents_selected = Table.SelectColumns(datetimeevents_expanded, {"itemid", "subject_id", "hadm_id", "icustay_id", "event_timestamp", "label", "linksto"}),

    // ---- INPUTEVENTS_MV ----
    inputevents_renamed = Table.RenameColumns(inputevents_mv, {{"starttime", "event_timestamp"}}),
    inputevents_merged = Table.NestedJoin(inputevents_renamed, {"itemid"}, d_items, {"itemid"}, "d_items", JoinKind.LeftOuter),
    inputevents_expanded = Table.ExpandTableColumn(inputevents_merged, "d_items", {"label", "linksto"}, {"label", "linksto"}),
    inputevents_selected = Table.SelectColumns(inputevents_expanded, {"itemid", "subject_id", "hadm_id", "icustay_id", "event_timestamp", "label", "linksto"}),

    // ---- OUTPUTEVENTS ----
    outputevents_renamed = Table.RenameColumns(outputevents, {{"charttime", "event_timestamp"}}),
    outputevents_merged = Table.NestedJoin(outputevents_renamed, {"itemid"}, d_items, {"itemid"}, "d_items", JoinKind.LeftOuter),
    outputevents_expanded = Table.ExpandTableColumn(outputevents_merged, "d_items", {"label", "linksto"}, {"label", "linksto"}),
    outputevents_selected = Table.SelectColumns(outputevents_expanded, {"itemid", "subject_id", "hadm_id", "icustay_id", "event_timestamp", "label", "linksto"}),

    // ---- PROCEDUREEVENTS_MV ----
    procedureevents_renamed = Table.RenameColumns(procedureevents_mv, {{"starttime", "event_timestamp"}}),
    procedureevents_merged = Table.NestedJoin(procedureevents_renamed, {"itemid"}, d_items, {"itemid"}, "d_items", JoinKind.LeftOuter),
    procedureevents_expanded = Table.ExpandTableColumn(procedureevents_merged, "d_items", {"label", "linksto"}, {"label", "linksto"}),
    procedureevents_selected = Table.SelectColumns(procedureevents_expanded, {"itemid", "subject_id", "hadm_id", "icustay_id", "event_timestamp", "label", "linksto"}),

    // ---- MICROBIOLOGYEVENTS (3 joins on different itemid fields) ----
    microbio_renamed = Table.RenameColumns(microbiologyevents, {{"charttime", "event_timestamp"}}),

    spec_join = Table.NestedJoin(microbio_renamed, {"spec_itemid"}, d_items, {"itemid"}, "spec_info", JoinKind.LeftOuter),
    spec_expanded = Table.ExpandTableColumn(spec_join, "spec_info", {"label", "linksto"}, {"label", "linksto"}),
    spec_selected = Table.SelectColumns(spec_expanded, {"spec_itemid", "subject_id", "hadm_id", "event_timestamp", "label", "linksto"}),
    spec_renamed = Table.RenameColumns(spec_selected, {{"spec_itemid", "itemid"}}),

    org_join = Table.NestedJoin(microbio_renamed, {"org_itemid"}, d_items, {"itemid"}, "org_info", JoinKind.LeftOuter),
    org_expanded = Table.ExpandTableColumn(org_join, "org_info", {"label", "linksto"}, {"label", "linksto"}),
    org_selected = Table.SelectColumns(org_expanded, {"org_itemid", "subject_id", "hadm_id", "event_timestamp", "label", "linksto"}),
    org_renamed = Table.RenameColumns(org_selected, {{"org_itemid", "itemid"}}),

    ab_join = Table.NestedJoin(microbio_renamed, {"ab_itemid"}, d_items, {"itemid"}, "ab_info", JoinKind.LeftOuter),
    ab_expanded = Table.ExpandTableColumn(ab_join, "ab_info", {"label", "linksto"}, {"label", "linksto"}),
    ab_selected = Table.SelectColumns(ab_expanded, {"ab_itemid", "subject_id", "hadm_id", "event_timestamp", "label", "linksto"}),
    ab_renamed = Table.RenameColumns(ab_selected, {{"ab_itemid", "itemid"}}),

    // ---- Combine all enriched tables ----
    combined = Table.Combine({
        chartevents_selected,
        datetimeevents_selected,
        inputevents_selected,
        outputevents_selected,
        procedureevents_selected,
        spec_renamed,
        org_renamed,
        ab_renamed
    }),

    // Detect data type
    ChangedcolumnType = Table.TransformColumnTypes(combined, {{"itemid", Int64.Type}, {"subject_id", Int64.Type}, {"hadm_id", Int64.Type}, {"icustay_id", Int64.Type}, {"event_timestamp", type datetime}, {"label", type text}, {"linksto", type text}}), 

    // Keep only first 10000 rows
    limited = Table.FirstN(ChangedcolumnType, 10000)
in
    limited

In [ ]:
# Load the event log CSV into a Pandas DataFrame
eventlog_path = "/Users/alejandromateocobo/Documents/PythonProjects/Integration_Of_LLMs_And_Process_Mining/data/mimic-iii-clinical-database-demo-1.4/c_EVENTLOG.csv"
df_eventlog = pd.read_csv(eventlog_path, sep=";")

# Show the first 5 rows of the event log
df_eventlog.head()

## 5. Prepare Event Log for PM4PY

In this step, the dataset is reformatted to match the structure expected by PM4PY, where each event log requires a case identifier, an activity name, and a timestamp. The data is then converted into a PM4PY event log object, which enables process mining algorithms to analyze the sequence of events across cases.

In [ ]:
# Rename columns for PM4PY
df_eventlog = df_eventlog.rename(columns={
    "hadm_id": "case:concept:name",
    "event_timestamp": "time:timestamp",
    "label": "concept:name"
})

# Convert the timestamp column to datetime
df_eventlog["time:timestamp"] = pd.to_datetime(df_eventlog["time:timestamp"], errors="coerce")

# Use PM4PY utility to ensure the dataframe is correctly formatted
df_eventlog = dataframe_utils.convert_timestamp_columns_in_df(df_eventlog)

# Convert the dataframe to an event log object
event_log = log_converter.apply(df_eventlog)

# Print basic statistics
print(f"Total cases: {len(set(df_eventlog['case:concept:name']))}")
print(f"Total events: {len(df_eventlog)}")

# Preview the formatted event log
df_eventlog.head()

## 6. Discover and Visualize the Directly-Follows Graph (DFG) with PM4PY

In this step, the frequency-based Directly-Follows Graph (DFG) is discovered from the event log using PM4PY and visualized to understand the control-flow structure of the process. The DFG is then converted into a readable textual format to prepare it for further analysis with a Large Language Model (LLM).

In [ ]:
# Discover the frequency-based DFG from the event log
dfg = dfg_discovery.apply(event_log)

# Visualize the DFG
gviz = dfg_vis.apply(dfg, variant=dfg_vis.Variants.FREQUENCY)
dfg_vis.view(gviz)

In [ ]:
def create_textual_dfg(dfg):
    lines = []
    for (a, b), freq in dfg.items():
        line = f"{a} → {b} (frequency = {freq})"
        lines.append(line)
    return "\n".join(lines)

# Convert the DFG to text
dfg_text = create_textual_dfg(dfg)

# Save it as a .txt file
with open("/Users/alejandromateocobo/Documents/PythonProjects/Integration_Of_LLMs_And_Process_Mining/data/context/dfg_text.txt", "w", encoding="utf-8") as file:
    file.write(dfg_text)

# Print the DFG in textual format
print(dfg_text)

## 7. Send the DFG to ChatGPT via OpenAI API

In [ ]:
import pinecone
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings

In [ ]:
load_dotenv()

pinecone_api_key = os.getenv("PINECONE_API_KEY")

# Initialize Pinecone
pinecone.init(api_key=pinecone_api_key)



In [87]:
# Initialize the GPT model
# llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

# conversation_buf = ConversationChain(
#     llm=llm,
#     memory=ConversationBufferMemory()
# )

# conversation = ConversationChain(llm=llm)

# Provide dataset description
dataset_description = """
Abstract
MIMIC-III is a large, freely-available database comprising deidentified health-related data associated with over forty thousand patients who stayed in critical care units of the Beth Israel Deaconess Medical Center between 2001 and 2012. The database includes information such as demographics, vital sign measurements made at the bedside (~1 data point per hour), laboratory test results, procedures, medications, caregiver notes, imaging reports, and mortality (including post-hospital discharge). MIMIC supports a diverse range of analytic studies spanning epidemiology, clinical decision-rule improvement, and electronic tool development. It is notable for three factors: it is freely available to researchers worldwide; it encompasses a diverse and very large population of ICU patients; and it contains highly granular data, including vital signs, laboratory results, and medications.

Background
In recent years there has been a concerted move towards the adoption of digital health record systems in hospitals. In the US, for example, the number of non-federal acute care hospitals with basic digital systems increased from 9.4 to 75.5% over the 7 year period between 2008 and 2014 [1]. Despite this advance, interoperability of digital systems remains an open issue, leading to challenges in data integration. As a result, the potential that hospital data offers in terms of understanding and improving care is yet to be fully realized. In parallel, the scientific research community is increasingly coming under criticism for the lack of reproducibility of studies [2].

Methods
MIMIC-III integrates deidentified, comprehensive clinical data of patients admitted to the Beth Israel Deaconess Medical Center in Boston, Massachusetts, and makes it widely accessible to researchers internationally under a data use agreement. The open nature of the data allows clinical studies to be reproduced and improved in ways that would not otherwise be possible. The MIMIC-III database was populated with data that had been acquired during routine hospital care, so there was no associated burden on caregivers and no interference with their workflow. Data was downloaded from several sources, including: archives from critical care information systems. hospital electronic health record databases. Social Security Administration Death Master File. Two different critical care information systems were in place over the data collection period: Philips CareVue Clinical Information System (models M2331A and M1215A; Philips Health-care, Andover, MA) and iMDsoft MetaVision ICU (iMDsoft, Needham, MA). These systems were the source of clinical data such as: time-stamped nurse-verified physiological measurements (for example, hourly documentation of heart rate, arterial blood pressure, or respiratory rate); documented progress notes by care providers; continuous intravenous drip medications and fluid balances. With exception to data relating to fluid intake, which differed significantly in structure between the CareVue and MetaVision systems, data was merged when building the database tables. Data which could not be merged is given a suffix to denote the data source. For example, inputs for patients monitored with the CareVue system are stored in INPUTEVENTS_CV, whereas inputs for patients monitored with the Metavision system are stored in INPUTEVENTS_MV. Additional information was collected from hospital and laboratory health record systems, including: patient demographics and in-hospital mortality. laboratory test results (for example, hematology, chemistry, and microbiology results). discharge summaries and reports of electrocardiogram and imaging studies. billing-related information such as International Classification of Disease, 9th Edition (ICD-9) codes, Diagnosis Related Group (DRG) codes, and Current Procedural Terminology (CPT) codes. Out-of-hospital mortality dates were obtained using the Social Security Administration Death Master File. Before data was incorporated into the MIMIC-III database, it was first deidentified in accordance with Health Insurance Portability and Accountability Act (HIPAA) standards using structured data cleansing and date shifting. The deidentification process for structured data required the removal of all eighteen of the identifying data elements listed in HIPAA, including fields such as patient name, telephone number, address, and dates. In particular, dates were shifted into the future by a random offset for each individual patient in a consistent manner to preserve intervals, resulting in stays which occur sometime between the years 2100 and 2200. Time of day, day of the week, and approximate seasonality were conserved during date shifting. Dates of birth for patients aged over 89 were shifted to obscure their true age and comply with HIPAA regulations: these patients appear in the database with ages of over 300 years. Protected health information was removed from free text fields, such as diagnostic reports and physician notes, using a rigorously evaluated deidentification system based on extensive dictionary look-ups and pattern-matching with regular expressions. The components of this deidentification system are continually expanded as new data is acquired. The project was approved by the Institutional Review Boards of Beth Israel Deaconess Medical Center (Boston, MA) and the Massachusetts Institute of Technology (Cambridge, MA). Requirement for individual patient consent was waived because the project did not impact clinical care and all protected health information was deidentified.

Data Description
MIMIC-III is a relational database consisting of 26 tables. Tables are linked by identifiers which usually have the suffix ‘ID’. For example, SUBJECT_ID refers to a unique patient, HADM_ID refers to a unique admission to the hospital, and ICUSTAY_ID refers to a unique admission to an intensive care unit. Charted events such as notes, laboratory tests, and fluid balance are stored in a series of ‘events’ tables. For example the OUTPUTEVENTS table contains all measurements related to output for a given patient, while the LABEVENTS table contains laboratory test results for a patient. Tables prefixed with ‘D_’ are dictionary tables and provide definitions for identifiers. For example, every row of CHARTEVENTS is associated with a single ITEMID which represents the concept measured, but it does not contain the actual name of the measurement. By joining CHARTEVENTS and D_ITEMS on ITEMID, it is possible to identify the concept represented by a given ITEMID. Developing the MIMIC data model involved balancing simplicity of interpretation against closeness to ground truth. As such, the model is a reflection of underlying data sources, modified over iterations of the MIMIC database in response to user feedback. Care has been taken to avoid making assumptions about the underlying data when carrying out transformations, so MIMIC-III closely represents the raw hospital data. Broadly speaking, five tables are used to define and track patient stays: ADMISSIONS; PATIENTS; ICUSTAYS; SERVICES; and TRANSFERS. Another five tables are dictionaries for cross-referencing codes against their respective definitions: D_CPT; D_ICD_DIAGNOSES; D_ICD_PROCEDURES; D_ITEMS; and D_LABITEMS. The remaining tables contain data associated with patient care, such as physiological measurements, caregiver observations, and billing information. In some cases it would be possible to merge tables—for example, the D_ICD_PROCEDURES and CPTEVENTS tables both contain detail relating to procedures and could be combined—but our approach is to keep the tables independent for clarity, since the data sources are significantly different. Rather than combining the tables within MIMIC data model, we suggest researchers develop database views and transforms as appropriate.
"""

# Save it as a .txt file
with open("/Users/alejandromateocobo/Documents/PythonProjects/Integration_Of_LLMs_And_Process_Mining/data/context/dataset_description.txt", "w", encoding="utf-8") as file:
    file.write(dataset_description)

# Send the dataset description as initial context to the chain
# conversation_buf(dataset_description)

# print(conversation_buf.memory.buffer)

In [ ]:
# 3. Send the DFG as additional context (memory will retain the dataset description)
conversation_buf(dfg_text)

print(conversation_buf.memory.buffer)

In [ ]:
# 4. Follow-up question using memory context (context is no longer needed here)
conversation_buf("Which activities seem to be the most frequent?")
print(conversation_buf.memory.buffer)

# System Prompt

pcsk_5GcY8X_QcmfM2dYCwS2H5nACFPPaf1BW9VVN919DdvUGk62YHWA7wNxA5gUHisv7fVTRvJ